<a href="https://colab.research.google.com/github/thnavear/Qualcomm-DL-Hackathon/blob/master/genai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/thnavear/Qualcomm-DL-Hackathon.git

In [2]:
from groq import Groq

client = Groq(api_key="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")
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Hello"}], # Added a user message
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


#os.environ["TOGETHER_API_KEY"]="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"

Hello. How can I assist you today?

In [4]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change

import pandas as pd
from groq import Groq

# Assuming you've already installed the groq library and cloned the repo
# !pip install groq
# !git clone https://github.com/thnavear/Qualcomm-DL-Hackathon.git

client = Groq(api_key="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")

try:
  df = pd.read_csv("/content/test1.csv")
except FileNotFoundError:
  print("Error: test.csv not found. Please upload the file to /content/")
  exit()

results = []

for index, row in df.iterrows():
    subject = row['ticket_subject']
    body = row['ticket_body']

    prompt = f"""Classify the following email into the given attributes.

Subject: {subject}
Body: {body}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority: The urgency level (low, medium, or high).
Language: The language code (en, de, etc.).
Type: The nature of the request (Incident, Request, Problem, or Change)

Return your response as a JSON object with the keys "Department", "Priority", "Language", and "Type"."""


    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,  # Lower temperature for more focused classifications
        max_completion_tokens=200,
        top_p=1,
        stream=False  # Set stream to False for better structured responses
    )

    response = completion.choices[0].message.content
    results.append(response)  # Append the classification result

# Now 'results' contains the JSON classification results for each email

# Example of printing the classifications (you might want to save them to a file)
for i, result in enumerate(results):
  print(f"Email {i+1} classification:\n{result}\n")

# Process results as needed (e.g., convert JSON strings to Python dicts, save to CSV)

Email 1 classification:
```json
{
  "Department": "IT Support",
  "Priority": "high",
  "Language": "it",
  "Type": "Incident"
}
```

Email 2 classification:
```json
{
  "Department": "Billing and Payments",
  "Priority": "medium",
  "Language": "de",
  "Type": "Incident"
}
```

Email 3 classification:
```json
{
  "Department": "Technical Support",
  "Priority": "medium",
  "Language": "fr",
  "Type": "Problem"
}
```

Email 4 classification:
```json
{
  "Department": "Billing and Payments",
  "Priority": "medium",
  "Language": "it",
  "Type": "Request"
}
```

Email 5 classification:
```json
{
  "Department": "Technical Support",
  "Priority": "high",
  "Language": "it",
  "Type": "Problem"
}
```

Email 6 classification:
```json
{
  "Department": "Product Support",
  "Priority": "medium",
  "Language": "it",
  "Type": "Incident"
}
```

Email 7 classification:
```json
{
  "Department": "IT Support",
  "Priority": "high",
  "Language": "it",
  "Type": "Incident"
}
```

Email 8 classifica